# Why and When Self-Attention Matters in Reinforcement Learning
> "Using RLlib and AttentionNet to master environments with stateless observations, here, stateless CartPole."

- hide: true
- toc: true
- branch: master
- badges: true
- comments: true
- categories: [python, ray, rllib, tensorflow, machine learning, reinforcement learning, attention]
- image: images/cartpole.jpg

TODOs:
* Image and short explanation of (self-)attention.
* Proper description of different options


In reinforcement learning (RL), the RL agent typically selects a suitable action based on the last observation.
Since many practical environments are stateful, this state should be taken into account when selecting an action.
As an example, consider the popular [OpenAI Gym CartPole environment](https://gym.openai.com/envs/CartPole-v1/),
where the task is to move a cart left or right in order to balance a pole on the cart as long as possible.

![OpenAI Gym CartPole-v1 Environment](attention/cartpole.gif "OpenAI Gym CartPole-v1 Environment")

Whether the cart should be moved left or right clearly depends on how the pole is currently moving,
i.e., in which direction it is swinging and with which velocity.
In this example, the pole's movement and velocity are an important part of the state,
which should determine the selected action (left or right).

In the following, I show different options how to deal with this state.

* State explicitly observed
* State not observed at all
* Sequence of last observations
* Sequence of last observations + attention

For each case, I use PPO with default hyperparamters.

> Tip: Also check out the [RLlib example using AttentionNet](https://github.com/ray-project/ray/blob/master/rllib/examples/attention_net.py).


## Setup

Install Ray RLlib and TensorFlow (also works with PyTorch); seaborn for plotting.
I am using Python 3.8 on Windows 10.

In [2]:
#collapse-output
!pip install ray[rllib]==1.8.0
!pip install tensorflow==2.7.0
!pip install seaborn==0.11.2

Import relevant dependencies and set constants.

In [13]:
import ray
import ray.tune
from ray.rllib.agents import ppo

# adjust num_cpus and num_gpus to your system
# for some reason, num_cpus=2 gets stuck on my system (when trying to train)
ray.init(num_cpus=3, ignore_reinit_error=True)

# stop conditions based on training iterations (each with 4000 train steps)
stop = {"training_iteration": 5}

2021-11-16 18:20:30,686	INFO worker.py:832 -- Calling ray.init() again after it has already been called.


## Explicitly Observe State (Velocity)


The default `CartPole-v1` environment has four observations as described [here](https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py#L26):
Cart position, cart velocity, pole angle, and pole angular velocity

In [14]:
import gym

env = gym.make("CartPole-v1")
env.observation_space

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)

In [15]:
#collapse-output

# load the PPO default config
config = ppo.DEFAULT_CONFIG.copy()
# use the normal, default CartPole-v1 envrion
config["env"] = "CartPole-v1"

# train; this takes a while
results = ray.tune.run("PPO", config=config, stop=stop)
print("Training finished successfully")

Trial name,status,loc
PPO_CartPole-v1_82e0a_00000,PENDING,


(pid=None) c:\users\stefan\git-repos\private\blog\venv\lib\site-packages\redis\connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
(pid=None)   warnings.warn(msg)


Trial name,status,loc
PPO_CartPole-v1_82e0a_00000,PENDING,


(pid=17188) 2021-11-16 18:20:49,461	INFO trainer.py:753 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
(pid=17188) 2021-11-16 18:20:49,462	INFO ppo.py:166 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(pid=17188) 2021-11-16 18:20:49,462	INFO trainer.py:770 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=None) c:\users\stefan\git-repos\private\blog\venv\lib\site-packages\redis\connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
(pid=None)   warnings.warn(msg)
(pid=None) c:\users\stefan\git-repos\private\blog\venv\lib\site-packages\redis\connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
(pid=None)   warnings.warn(msg)
(pid=17172) 2021-11-16 18:21:03,326	WARNING deprecation.py:38 -- Deprec

Trial name,status,loc
PPO_CartPole-v1_82e0a_00000,RUNNING,127.0.0.1:17188


...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_CartPole-v1_82e0a_00000,RUNNING,127.0.0.1:17188,4,35.0285,16000,97.77,425,10,97.77


Result for PPO_CartPole-v1_82e0a_00000:
  agent_timesteps_total: 20000
  custom_metrics: {}
  date: 2021-11-16_18-21-49
  done: true
  episode_len_mean: 133.09
  episode_media: {}
  episode_reward_max: 500.0
  episode_reward_mean: 133.09
  episode_reward_min: 10.0
  episodes_this_iter: 12
  episodes_total: 333
  experiment_id: 95780d6ad71a4a26be1d679814f7cb8b
  hostname: nb-stschn
  info:
    learner:
      default_policy:
        custom_metrics: {}
        learner_stats:
          cur_kl_coeff: 0.30000001192092896
          cur_lr: 4.999999873689376e-05
          entropy: 0.5628640651702881
          entropy_coeff: 0.0
          kl: 0.0053915660828351974
          model: {}
          policy_loss: -0.012018542736768723
          total_loss: 663.389892578125
          vf_explained_var: 0.19897815585136414
          vf_loss: 663.4002685546875
    num_agent_steps_sampled: 20000
    num_agent_steps_trained: 20000
    num_steps_sampled: 20000
    num_steps_trained: 20000
    num_steps_train

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_CartPole-v1_82e0a_00000,TERMINATED,127.0.0.1:17188,5,43.2415,20000,133.09,500,10,133.09


(pid=17188) [2021-11-16 18:21:50,089 E 17188 8580] raylet_client.cc:159: IOError: Unknown error [RayletClient] Failed to disconnect from raylet.
(pid=17188) Windows fatal exception: access violation
(pid=17188) 
(pid=17172) [2021-11-16 18:21:50,106 C 17172 1512] core_worker.cc:2234:  Check failed: _s.ok() Bad status: IOError: Unknown error
(pid=17172) *** StackTrace Information ***
(pid=17172)     PyInit__raylet
(pid=17172)     PyInit__raylet
(pid=17172)     PyInit__raylet
(pid=17172)     PyInit__raylet
(pid=17172)     PyInit__raylet
(pid=17172)     PyInit__raylet
(pid=17172)     PyInit__raylet
(pid=17172)     PyInit__raylet
(pid=17172)     PyInit__raylet
(pid=17172)     PyInit__raylet
(pid=17172)     PyInit__raylet
(pid=17172)     PyInit__raylet
(pid=17172)     PyInit__raylet
(pid=17172)     PyInit__raylet
(pid=17172)     PyInit__raylet
(pid=17172)     PyNumber_InPlaceLshift
(pid=17172)     Py_CheckFunctionResult
(pid=17172)     PyEval_EvalFrameDefault
(pid=17172)     Py_CheckFunction

Training finished successfully


In [16]:
# check results
results.default_metric = "episode_reward_mean"
results.default_mode = "max"
# print the mean episode reward = episode length --> higher = better
results.best_result["episode_reward_mean"]

133.09

In [25]:
results.dataframe()

,episode_reward_max,episode_reward_min,episode_reward_mean,episode_len_mean,episodes_this_iter,num_healthy_workers,timesteps_total,timesteps_this_iter,agent_timesteps_total,done,...,config/synchronize_filters,config/tf_session_args,config/timesteps_per_iteration,config/train_batch_size,config/use_critic,config/use_gae,config/vf_clip_param,config/vf_loss_coeff,config/vf_share_layers,logdir
0,500.0,10.0,133.09,133.09,12,2,20000,0,20000,True,...,True,"{'allow_soft_placement': True, 'device_count':...",0,4000,True,True,10.0,1.0,-1,C:\Users\Stefan\ray_results\PPO\PPO_CartPole-v...


In [ ]:

import seaborn as sns


eps_lengths = results.best_result["hist_stats"]["episode_lengths"]
eps = [i for i in range(len(eps_lengths))]
sns.scatterplot(eps, eps_lengths)

## Do Not Observe Velocity

In many practical

In [ ]:
from ray.rllib.examples.env.stateless_cartpole import StatelessCartPole
from ray.tune import registry

registry.register_env("StatelessCartPole", lambda _: StatelessCartPole())
config["env"] = "StatelessCartPole"
# train; this takes a while
results = ray.tune.run("PPO", config=config, stop=stop)
print("Training finished successfully")

Trial name,status,loc
PPO_StatelessCartPole_38798_00000,PENDING,


(pid=None) c:\users\stefan\git-repos\private\blog\venv\lib\site-packages\redis\connection.py:77: UserWarning: redis-py works best with hiredis. Please consider installing
(pid=None)   warnings.warn(msg)


Trial name,status,loc
PPO_StatelessCartPole_38798_00000,PENDING,


(pid=7440) 2021-11-16 18:33:04,342	INFO trainer.py:753 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
(pid=7440) 2021-11-16 18:33:04,342	INFO ppo.py:166 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(pid=7440) 2021-11-16 18:33:04,342	INFO trainer.py:770 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
